# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [44]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [29]:
!pip install yelpapi

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [45]:
# Load API Credentials
with open('C:\\Users\\sheha\\.secret\\yelp_api.json', 'r') as f:
    login =json.load(f)

In [46]:
login.keys

<function dict.keys>

In [47]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [48]:
# set our API call parameters and filename before the first call
location = 'Seattle,WA 98122'
term = 'pizza'

In [49]:
## Specify fodler for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)


In [50]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+location.split(',')[0]+'_'+term+'.json'

In [51]:
JSON_FILE

'Data/Seattle_pizza.json'

### Check if Json File exists and Create it if it doesn't

In [52]:
## Check if JSON_FILE exists
file_exist =os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exist == False:    
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder) > 0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists")
    


[i] Data/Seattle_pizza.json already exists


### Load JSON FIle and account for previous results

In [53]:
## Load previous results and use len of results for offset

## set offset based on previous results


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [55]:
# use our yelp_api variable's search_query method to perform our API call
results= yelp.search_query(term =term, location = location)

In [56]:
type(results)

dict

In [58]:
## How many results total?
len(results)

3

In [60]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [61]:
results['businesses'][0]

{'id': 't-Z9bvmlgUyDtomGmttrUQ',
 'alias': 'guerilla-pizza-kitchen-seattle',
 'name': 'Guerilla Pizza Kitchen',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEjc9J_4Tb1PtAb71UA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/guerilla-pizza-kitchen-seattle?adjust_creative=WtdOX25IrxJjIjuZZCkBjQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=WtdOX25IrxJjIjuZZCkBjQ',
 'review_count': 1,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'popuprestaurants', 'title': 'Pop-Up Restaurants'}],
 'rating': 5.0,
 'coordinates': {'latitude': 47.608127, 'longitude': -122.302435},
 'transactions': [],
 'location': {'address1': '2300 E Cherry St',
  'address2': '',
  'address3': None,
  'city': 'Seattle',
  'zip_code': '98122',
  'country': 'US',
  'state': 'WA',
  'display_address': ['2300 E Cherry St', 'Seattle, WA 98122']},
 'phone': '',
 'display_phone': '',
 'distance': 749.1731129459927}

In [62]:
results['total']

859

In [63]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113,NaN
1,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,100,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[pickup, delivery]","{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393,NaN
2,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,40,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1686.862176,NaN
3,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,360,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011,$$
4,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,140,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,8784.075722,$$
5,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,342,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[pickup, delivery]","{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796,$$
6,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,962,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",[delivery],"{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,2080.359405,$
7,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,205,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 47.598976, 'longitude': -122.294685}","[pickup, delivery]","{'address1': '2901 S Jackson St', 'address2': ...",+12066026333,(206) 602-6333,1571.867438,$$
8,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,180,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.636044, 'longitude': -122.277377}",[delivery],"{'address1': '4235 E Madison St', 'address2': ...",+12068606110,(206) 860-6110,2897.252844,$$
9,Ss_66zV-zhK_57_6MgV93w,aglio-e-olio-seattle,Aglio e Olio,https://s3-media2.fl.yelpcdn.com/bphoto/inrSl1...,False,https://www.yelp.com/biz/aglio-e-olio-seattle?...,272,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 47.62328027309258, 'longitude': -...",[delivery],"{'address1': '2743 E Madison St', 'address2': ...",,,1132.743207,$$


- Where is the actual data we want to save?

In [64]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [68]:
# Use math.ceil to round up for the total number of pages of results.
#import additional packages for control our loop
import time, math

# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(results['total']/results_per_page)
n_pages

43

In [69]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location =location, 
                                    term =term,
                                    offset = n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results,f)
                    
    ## What to do if we get an error/exception.
    except Exception as e: #saving the error message so we can print it
        print('[!] ERROR:', e)


  0%|          | 0/43 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [71]:
df = pd.read_json(JSON_FILE)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113,NaN
1,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,100,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[pickup, delivery]","{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393,NaN
2,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,40,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1686.862176,NaN
3,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,360,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011,$$
4,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,140,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,8784.075722,$$


In [72]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle_pizza.csv.gz'

In [73]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression= 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [74]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 837,966 Bytes
CSV.GZ FILE: 121,432 Bytes
the csv.gz is 6.900701627248171 times smaller!


## Next Class: Processing the Results and Mapping 